In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 2s (143 kB/s)
Reading package lists... Done


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)

home_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=',', header=True)

home_df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [ ]:
# 2. Create a temporary view of the DataFrame.
home_df.createOrReplaceTempView('homes')

In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
SELECT date_built, round(avg(price),2) AS Average FROM homes
WHERE bedrooms=4
GROUP BY date_built
ORDER BY date_built DESC
""").show()


+----------+---------+
|date_built|  Average|
+----------+---------+
|      2017|296576.69|
|      2016|296050.24|
|      2015|307908.86|
|      2014|299073.89|
|      2013|299999.39|
|      2012|298233.42|
|      2011| 302141.9|
|      2010|296800.75|
+----------+---------+



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
SELECT date_built, round(avg(price),2) AS Average FROM homes
WHERE bedrooms=3 AND bathrooms=3
GROUP BY date_built
ORDER BY date_built DESC
""").show()


+----------+---------+
|date_built|  Average|
+----------+---------+
|      2017|292676.79|
|      2016|290555.07|
|      2015| 288770.3|
|      2014|290852.27|
|      2013|295962.27|
|      2012|293683.19|
|      2011|291117.47|
|      2010|292859.62|
+----------+---------+



In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
SELECT date_built, round(avg(price),2) AS Average FROM homes
WHERE bedrooms=3 AND bathrooms=3 AND floors=2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built DESC
""").show()



+----------+---------+
|date_built|  Average|
+----------+---------+
|      2017|280317.58|
|      2016| 293965.1|
|      2015|297609.97|
|      2014|298264.72|
|      2013|303676.79|
|      2012|307539.97|
|      2011|276553.81|
|      2010|285010.22|
+----------+---------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT date_built, round(avg(price),2) As Average, round(avg(view),2) AS View FROM homes
WHERE price >=350000
GROUP BY date_built
ORDER BY date_built DESC
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------+---------+-----+
|date_built|  Average| View|
+----------+---------+-----+
|      2017|475884.72|32.03|
|      2016|478550.09|32.91|
|      2015|471517.37|31.89|
|      2014|475674.82|32.28|
|      2013|476837.88|32.61|
|      2012|475524.98|33.48|
|      2011|472861.91|32.28|
|      2010|463301.08|30.64|
+----------+---------+-----+

--- 0.3537180423736572 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table homes")

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('homes')

True

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
SELECT date_built, round(avg(price),2) As Average, round(avg(view),2) AS View FROM homes
WHERE price >=350000
GROUP BY date_built
ORDER BY date_built DESC
""").show()


print("--- %s seconds ---" % (time.time() - start_time))


+----------+---------+-----+
|date_built|  Average| View|
+----------+---------+-----+
|      2017|475884.72|32.03|
|      2016|478550.09|32.91|
|      2015|471517.37|31.89|
|      2014|475674.82|32.28|
|      2013|476837.88|32.61|
|      2012|475524.98|33.48|
|      2011|472861.91|32.28|
|      2010|463301.08|30.64|
+----------+---------+-----+

--- 0.19885993003845215 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
home_df.write.partitionBy("date_built").mode("overwrite").parquet('homes_par')

In [ ]:
# 11. Read the parquet formatted data.
p_home_df=spark.read.parquet('homes_par')

In [ ]:
# 12. Create a temporary table for the parquet data.
p_home_df.createOrReplaceTempView('home_p')

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
SELECT date_built, round(avg(price),2) As Average, round(avg(view),2) AS View FROM home_p
WHERE price >=350000
GROUP BY date_built
ORDER BY date_built DESC
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----------+---------+-----+
|date_built|  Average| View|
+----------+---------+-----+
|      2017|475884.72|32.03|
|      2016|478550.09|32.91|
|      2015|471517.37|31.89|
|      2014|475674.82|32.28|
|      2013|476837.88|32.61|
|      2012|475524.98|33.48|
|      2011|472861.91|32.28|
|      2010|463301.08|30.64|
+----------+---------+-----+

--- 0.484637975692749 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table homes")

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("homes")


False